In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [22]:
df = pd.read_pickle('../wiki_data.pkl')

df.sample()

In [41]:
X = df['content']
y = df['category']

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(min_df = 2, stop_words = 'english', ngram_range=(1,2))
document_term_matrix_sps = tfidf_vectorizer.fit_transform(X)
document_term_matrix_df = pd.DataFrame(document_term_matrix_sps.toarray(),
                                       index=X.index,
                                       columns=tfidf_vectorizer.get_feature_names())

document_term_matrix_df.shape

In [45]:
from sklearn.decomposition import TruncatedSVD

n_components = 300
SVD = TruncatedSVD(n_components)
svd_matrix = SVD.fit_transform(document_term_matrix_sps)


In [53]:
!pip install xgboost

    100% |████████████████████████████████| 1.2MB 538kB/s eta 0:00:01
  Running setup.py bdist_wheel for xgboost ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/5e/c1/d6/522af54e5cc001fad4dd855117f8bf61b11d56443e06672e26
Successfully built xgboost


In [62]:
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import GridSearchCV

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [59]:
pipe = Pipeline([
    ('vectorize', TfidfVectorizer(min_df = 2, stop_words = 'english', ngram_range=(1,2))),
    ('svd', TruncatedSVD(n_components=300)),
    ('pred_model', XGBClassifier())
])

In [67]:
params= {
    'svd__n_components': [300, 325, 350]
}

In [68]:
gs = GridSearchCV(pipe, param_grid=params)

In [69]:
gs.fit(X_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(steps=[('vectorize', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=2,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=Tru...logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'svd__n_components': [300, 325, 350]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [70]:
gs.score(X_test, y_test)

0.93939393939393945